# Import Packages

In [1]:
!nvidia-smi

Wed May 25 00:52:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3D:00.0 Off |                    0 |
| N/A   29C    P0    66W / 300W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip -q install gdown

In [3]:
import os
import glob
import numpy as np
import random
import cv2
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import json
import torch
import shutil

from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import torch.nn.functional as F

import torch
import numpy as np
import segmentation_models_pytorch as smp
from torch import nn

In [4]:
myseed = 2022
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Preprocessing

## Mask

In [7]:
dataset_root = './Data/SEG_Train_Datasets'
anno_path = os.path.join(dataset_root, 'Train_Annotations')

mask_path = os.path.join(dataset_root, 'Train_Masks')
os.makedirs(os.path.join(dataset_root, 'Train_Masks'), exist_ok=True)
for jsonfile in os.listdir(anno_path):
    f = open(os.path.join(anno_path, jsonfile))
    data = json.load(f)
    mask = np.zeros((data['imageHeight'], data['imageWidth'], 1), dtype=np.uint8)
    for polygan in data['shapes']:
        pts = np.array(polygan['points'], dtype=np.int32)
        cv2.fillPoly(mask, [pts], color=255)
    save_mask_path = jsonfile.split('.')[0] + '.png'
    save_mask_path = os.path.join(mask_path, save_mask_path)
    cv2.imwrite(save_mask_path, mask)

## Split train, valid and test

In [8]:
os.makedirs(os.path.join(dataset_root, 'Fold1_Images'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'Fold1_Masks'), exist_ok=True)

os.makedirs(os.path.join(dataset_root, 'Fold2_Images'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'Fold2_Masks'), exist_ok=True)

os.makedirs(os.path.join(dataset_root, 'Fold3_Images'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'Fold3_Masks'), exist_ok=True)

os.makedirs(os.path.join(dataset_root, 'Fold4_Images'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'Fold4_Masks'), exist_ok=True)

os.makedirs(os.path.join(dataset_root, 'Test_Images'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'Test_Masks'), exist_ok=True)

In [9]:
img_list = sorted(os.listdir(os.path.join(dataset_root, 'Train_Images')))

In [10]:
num = len(os.listdir(os.path.join(dataset_root, 'Train_Images')))
for i in range(4, len(img_list), 5):
    img = img_list[i-4]
    shutil.move(dataset_root+'/Train_Images/'+img, dataset_root+'/Fold1_Images')
    shutil.move(dataset_root+'/Train_Masks/'+img.replace('.jpg','.png'), dataset_root+'/Fold1_Masks')
    
    img = img_list[i-3]
    shutil.move(dataset_root+'/Train_Images/'+img, dataset_root+'/Fold2_Images')
    shutil.move(dataset_root+'/Train_Masks/'+img.replace('.jpg','.png'), dataset_root+'/Fold2_Masks')

    img = img_list[i-2]
    shutil.move(dataset_root+'/Train_Images/'+img, dataset_root+'/Fold3_Images')
    shutil.move(dataset_root+'/Train_Masks/'+img.replace('.jpg','.png'), dataset_root+'/Fold3_Masks')
    
    img = img_list[i-1]
    shutil.move(dataset_root+'/Train_Images/'+img, dataset_root+'/Fold4_Images')
    shutil.move(dataset_root+'/Train_Masks/'+img.replace('.jpg','.png'), dataset_root+'/Fold4_Masks')
    
    img = img_list[i]
    shutil.move(dataset_root+'/Train_Images/'+img, dataset_root+'/Test_Images')
    shutil.move(dataset_root+'/Train_Masks/'+img.replace('.jpg','.png'), dataset_root+'/Test_Masks')

In [11]:
for i in range(1050,1053):
    img = img_list[i]
    shutil.move(dataset_root+'/Train_Images/'+img, dataset_root+'/Test_Images')
    shutil.move(dataset_root+'/Train_Masks/'+img.replace('.jpg','.png'), dataset_root+'/Test_Masks')

In [6]:
print('Fold1 :',len(os.listdir(os.path.join(dataset_root, 'Fold1_Images'))))
print('Fold2 :',len(os.listdir(os.path.join(dataset_root, 'Fold2_Images'))))
print('Fold3 :',len(os.listdir(os.path.join(dataset_root, 'Fold3_Images'))))
print('Fold4 :',len(os.listdir(os.path.join(dataset_root, 'Fold4_Images'))))
print('Test set :',len(os.listdir(os.path.join(dataset_root, 'Test_Images'))))

Fold1 : 210
Fold2 : 210
Fold3 : 210
Fold4 : 210
Test set : 213
